### Data Prepocessing Step

In [1]:
import pandas as pd

In [2]:
import os

In [3]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification'

In [8]:
#update entity
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)# To ensure that we give correct type of input as specified for the data ingestion process.
class dataprocessingconfig:
    root_dir: Path
    drop_feature: list
    categorical_feature: list
    continuous_feature: list
    save_processed_data: Path
    downloaded_data: Path

In [9]:
#update configration manger
from AML_Classifier.constants.__init__ import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from AML_Classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_process_config(self) -> dataprocessingconfig:
        config = self.config.data_prepocess
        create_directories([config.root_dir])
        downloaded_data = os.path.join(self.config.data_ingestion.root_dir,"data.csv")

        data_process_config= dataprocessingconfig(
            root_dir = Path(config.root_dir),
            drop_feature=config.drop_feature,
            categorical_feature = config.categorical_feature,
            continuous_feature = config.continuous_feature,
            downloaded_data=Path(downloaded_data),
            save_processed_data=Path(config.save_processed_data)
            )
        return data_process_config

In [21]:
#update the compontents
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [23]:
class Processdata:
    def __init__(self,config: dataprocessingconfig):
        self.config = config

    def preprocessing(self):
        df=pd.read_csv(self.config.downloaded_data)
        #Drop the unnessary feature
        df=df.drop(self.config.drop_feature,axis=1)
        #Intialize a dataframe
        processed_df=pd.DataFrame()
        #Label encoding
        label_encoder = LabelEncoder()
        for i in list(self.config.categorical_feature):
            processed_df[i]=label_encoder.fit_transform(df[i])

        #Scaling the continous variable
        standard_scaler = StandardScaler()
        for i in list(self.config.continuous_feature):
            processed_df[i]=standard_scaler.fit_transform(df[[i]])

        #Save the file
        processed_df.to_csv(str(self.config.save_processed_data),index=False)

In [27]:
#pipeline
try:
    config = ConfigurationManager()
    get_data_process_config = config.get_data_process_config()
    Process_data = Processdata(config = get_data_process_config)
    Process_data.preprocessing()
except Exception as e:
    raise e

[2024-06-28 19:07:11,992: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-28 19:07:12,004: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 19:07:12,004: INFO: common: created directory at: artifacts]
[2024-06-28 19:07:12,022: INFO: common: created directory at: artifacts/model_data]
